In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import pandas as pd
import keras.utils as ku
import zipfile
import time

from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/MyDrive/DATASET.zip"
extract_path = "/content/DATASET"

if not os.path.exists(extract_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content")
    print("Dataset extracted successfully")
else:
    print("Dataset already extracted.")

train_dir = os.path.join(extract_path, "TRAIN")
test_dir = os.path.join(extract_path, "TEST")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

def build_model(hp):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    x = Flatten()(base_model.output)

    x = Dense(hp.Int('units_1', min_value=64, max_value=256, step=64), activation='relu')(x)
    x = Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1))(x)

    x = Dense(hp.Int('units_2', min_value=32, max_value=128, step=32), activation='relu')(x)
    x = Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1))(x)

    output_layer = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output_layer)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4, 3e-4, 1e-3])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='/content/drive/MyDrive/hyperparameter_tuning',
    project_name='waste_classifier_vgg16'
)

tuner.search(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)]
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

def classify_image(image_path):
    img = ku.load_img(image_path, target_size=(150, 150))
    img = ku.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = best_model.predict(img)[0][0]

    if prediction > 0.7:
        label = 'Recyclable'
    elif prediction < 0.3:
        label = 'Organic Waste'
    else:
        label = 'General Waste'

    print(f"Classified: {label} with probability {round(float(prediction), 2)}")

    plt.axis('off')
    plt.imshow(img.squeeze())
    plt.title(f"Prediction: {label}")
    plt.show()

model_save_path = "/content/drive/MyDrive/waste_classifier_VGG16_best.keras"
best_model.save(model_save_path)
print(f"Best model saved successfully at: {model_save_path}")



In [ ]:
saved_model_path = "/content/drive/MyDrive/waste_classifier_VGG16_best.keras"
model = tf.keras.models.load_model(saved_model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_path = "/content/drive/MyDrive/waste_classifier_Vgg16_besttflite.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved successfully at: {tflite_model_path}")


In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    average_precision_score
)
import matplotlib.pyplot as plt
import pandas as pd
import time
import zipfile

from google.colab import drive
drive.mount('/content/drive')

zip_file_path = "/content/drive/MyDrive/DATASET.zip"
model_path = "/content/drive/MyDrive/waste_classifier_VGG16_best.keras"

extract_path = "/content/DATASET"

print("Unzipping dataset")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content")
print("Unzipped successfully.")

test_dir = os.path.join(extract_path, "TEST")

model = tf.keras.models.load_model(model_path)
print("Keras model loaded successfully.")

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

pred_probs = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = (pred_probs > 0.5).astype(int).flatten()

report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print("\nClassification Report:\n")
print(report_df)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

map_score = average_precision_score(y_true, pred_probs.flatten())
print(f"\nMean Average Precision (mAP): {map_score:.4f}")

latency_times = []
test_generator.reset()
for _ in range(1):
    batch_images, _ = next(test_generator)
    start_time = time.time()
    _ = model.predict(batch_images)
    end_time = time.time()
    latency_times.append(end_time - start_time)

avg_latency_ms = np.mean(latency_times) * 1000
print(f"Latency: {avg_latency_ms:.2f} ms")

test_generator.reset()
start_time = time.time()
_ = model.predict(test_generator, steps=len(test_generator), verbose=0)
end_time = time.time()
num_images = len(test_generator.filenames)
fps = num_images / (end_time - start_time)
print(f"Throughput: {fps:.2f} FPS (Frames per second)")


In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_true, pred_probs)
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_true, pred_probs)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.hist(pred_probs, bins=50, alpha=0.7)
plt.xlabel("Predicted Probability")
plt.ylabel("Number of Samples")
plt.title("Confidence Distribution of Predictions")
plt.grid(True)
plt.show()